In [1]:
from cdb.core.component import *
from cdb.core.manip import *
from cdb.utils.node import *
import numpy as np
import scipy as sc
import sympy as sp
import pickle

In [2]:

metric:= { g_{t t} = -(1-2 M/r),   
       g_{r r} = 1/(1-2 M/r), 
       g_{\theta\theta} = r**2, 
       g_{\phi\phi}=r**2 \sin(\theta)**2
     };

${}\left[g_{t t} = -1+2M {r}^{-1},~ g_{r r} = {\left(1-2M {r}^{-1}\right)}^{-1},~ g_{\theta \theta} = {r}^{2},~ g_{\phi \phi} = {r}^{2} {\left(\sin{\theta}\right)}^{2}\right]$

In [3]:
class Met: 
    
    
    {r,t,\theta,\phi}::Coordinate.
    {\alpha, \beta, \mu,\nu,\rho,\sigma,\lambda,\kappa,\chi,\gamma}::Indices(values={t,r,\theta,\phi}, position=fixed).
    g_{\mu\nu}::Metric.
    g^{\mu\nu}::InverseMetric.
    {g_{\mu}^{\nu},g^{\mu}_{\nu}}::KroneckerDelta.
    \Gamma^{\alpha}_{\mu\nu}::TableauSymmetry(shape={2}, indices={1,2}).
    \partial{#}::PartialDerivative.
    
    _chris := \Gamma^{\mu}_{\nu\rho} -> 1/2 g^{\mu\sigma} ( \partial_{\rho}{g_{\nu\sigma}} +\partial_{\nu}{g_{\rho\sigma}} -\partial_{\sigma}{g_{\nu\rho}}).
    _riemann_tensor := R^{\rho}_{\sigma\mu\nu} = \partial_{\mu}{\Gamma^{\rho}_{\nu\sigma}} -\partial_{\nu}{\Gamma^{\rho}_{\mu\sigma}} 
                                              +\Gamma^{\rho}_{\mu\lambda} \Gamma^{\lambda}_{\nu\sigma} -\Gamma^{\rho}_{\nu\lambda} \Gamma^{\lambda}_{\mu\sigma}.
    _ricci_tensor      := R_{\sigma\nu} = R^{\rho}_{\sigma\rho\nu}.
    _ricci_scalar  := R = R_{\sigma\nu} g^{\sigma\nu}.
    _einstein_tensor   := G_{\mu\nu} = R_{\mu\nu} - 1/2 g_{\mu\nu} R.
    
    
    def __init__(self, metric):
        
        self._metric := @(metric).
        complete(self._metric, $g^{\mu\nu}$)
        self.__chris()
        self.__riemann_tensor()
        self.__ricci_tensor()
        self.__ricci_scalar()
        self.__einstein_tensor()
     
    # Private methods to calculate christoffel,...,einstein tensor
    
    def __chris(self):
        self._chris := \Gamma^{\mu}_{\nu\rho} = 1/2 g^{\mu\sigma} ( \partial_{\rho}{g_{\nu\sigma}} +\partial_{\nu}{g_{\rho\sigma}} -\partial_{\sigma}{g_{\nu\rho}}).
        evaluate(self._chris, self._metric, rhsonly=True)
        
    
    def __riemann_tensor(self):
        self._riemann_tensor := R^{\rho}_{\sigma\mu\nu} = \partial_{\mu}{\Gamma^{\rho}_{\nu\sigma}} -\partial_{\nu}{\Gamma^{\rho}_{\mu\sigma}} 
                                              +\Gamma^{\rho}_{\mu\lambda} \Gamma^{\lambda}_{\nu\sigma} -\Gamma^{\rho}_{\nu\lambda} \Gamma^{\lambda}_{\mu\sigma}.
        evaluate(substitute(self._riemann_tensor, self._chris), self._metric,rhsonly=True)
        
    def __ricci_tensor(self):
        self._ricci_tensor := R_{\sigma\nu} = R^{\rho}_{\sigma\rho\nu}.
        evaluate(substitute(self._ricci_tensor, self._riemann_tensor), self._metric,rhsonly=True)

    def __ricci_scalar(self):
        self._ricci_scalar  := R = R_{\sigma\nu} g^{\sigma\nu}.
        evaluate(substitute(self._ricci_scalar, self._ricci_tensor), self._metric,rhsonly=True)
        
    def __einstein_tensor(self):
        self._einstein_tensor := G_{\mu\nu} = R_{\mu\nu} - 1/2 g_{\mu\nu} R.
        substitute(self._einstein_tensor, self._ricci_tensor)
        substitute(self._einstein_tensor, self._ricci_scalar)
        evaluate(self._einstein_tensor, self._metric, rhsonly=True)
    
    
    
    def chris_component(self, ex1):
        if n_args(ex1) !=3:
            return print("3 indis giriniz")
        component = Ex(", ".join([nth_arg(ex1, 1)._latex_(), nth_arg(ex1, 2)._latex_(), nth_arg(ex1, 0)._latex_()]))
        comp = Ex(r"\Gamma^{" + nth_arg(ex1, 0)._latex_() + "}_{" + nth_arg(ex1, 1)._latex_() + " " + nth_arg(ex1, 2)._latex_()+"}")
        chr := \Gamma^{\mu}_{\nu\rho} -> @(comp).
        try:
            chris_comp = get_component(self._chris, component)
            substitute(chris_comp, chr)
        except:
            chris_comp := @(comp) = 0.
        return chris_comp
    
    def riemann_component(self, ex1):
        if n_args(ex1) !=4:
            return print("4 indis giriniz")
        component = Ex(", ".join([nth_arg(ex1, 3)._latex_(), nth_arg(ex1, 1)._latex_(), nth_arg(ex1, 0)._latex_(), nth_arg(ex1, 2)._latex_()]))
        comp = Ex(r"R^{" + nth_arg(ex1, 0)._latex_() + "}_{" + nth_arg(ex1, 1)._latex_() + " " + nth_arg(ex1, 2)._latex_() + " "+ nth_arg(ex1, 3)._latex_()+"}")
        rt := R^{\rho}_{\sigma\mu\nu} -> @(comp).
        try:
            riemann_comp = get_component(self._riemann, component)
            substitute(riemann_comp, rt)
        except:
            riemann_comp := @(comp) = 0.
        return riemann_comp
        
    def ricci_component(self, ex1):
        if n_args(ex1) !=2:
            return print("2 indis giriniz")
        comp = Ex(r"R_{" + nth_arg(ex1, 0)._latex_() + " " + nth_arg(ex1, 1)._latex_() +"}")
        rct := R_{\sigma \nu} -> @(comp).
        try:
            ricci_comp = get_component(self._ricci, component)
            substitute(ricci_comp, rct)
        except:
            ricci_comp := @(comp) = 0.
        return ricci_comp
        
    def chris_components_coord(self, ex):
        comp = Ex(r"g^{" + ex + "}_{" + ex +"}")
        g := [ @(comp) -> 1].
        chris_comp := tmp -> \Gamma^{\lambda}_{\nu\rho} g^{\mu}_{\lambda}.
        substitute(chris_comp,self._chris)
        evaluate(chris_comp,g)
        substitute(chris_comp, $tmp -> \Gamma^{\mu}_{\nu\rho}$)
        return chris_comp
        
    def riemann_components_coord(self, ex):
        comp = Ex(r"g^{" + ex + "}_{" + ex +"}")
        g := [ @(comp) -> 1].
        riemann_comp := tmp -> R^{\lambda}_{\sigma\mu\nu} g^{\rho}_{\lambda}.
        substitute(riemann_comp,self._riemann_tensor)
        evaluate(riemann_comp,g)
        substitute(riemann_comp, $tmp -> R^{\rho}_{\sigma\mu\nu}$)
        return riemann_comp
    

In [4]:
class Geodesic:

    {r,t,\phi,\theta}::Coordinate.
    {\alpha, \beta, \mu,\nu,\rho,\sigma,\lambda,\kappa,\chi,\gamma}::Indices(values={t,r,\phi,\theta}, position=fixed).
    g_{\mu\nu}::Metric.
    g^{\mu\nu}::InverseMetric.
    {\delta{#},g_{\mu}^{\nu},g^{\mu}_{\nu}}::KroneckerDelta.
    \Gamma^{\alpha}_{\mu\nu}::TableauSymmetry(shape={2}, indices={1,2}).
    \partial{#}::PartialDerivative.
    \nabla{#}::Derivative.
    
    _u := [
                u^{t} = u^{t},
                u^{r} = u^{r},
                u^{\theta} = u^{\theta},
                u^{\phi} = u^{\phi}
                ].
                                
    _ut1 := [
                 u^{t} = 1,
                 u^{r} = u^{r},
                 u^{\theta} = u^{\theta},
                 u^{\phi} = u^{\phi}
                 ].
                     
        
        
    _indice_to_number := [
                                u^{t} -> u0,
                                u^{r} -> u1,
                                u^{\theta} -> u2,
                                   u^{\phi} -> u3
                                  ].
        
    _proper_geodesic := K^{\mu} = \Gamma^{\mu}_{\alpha \beta} u^{\alpha} u^{\beta}.
    
    _coordinate_geodesic := a^{\mu} = -K^{\mu}+ K^{t} u^{\mu}.
    
    _k_to_a := K^{\mu} -> a^{\mu}.
    
    def __init__(self, chris):
        self._chris            := @(chris).
        
        self.__proper_geodesic()
        self.__coordinate_geodesic()
        self.__equatorial_proper_geodesic()
        self.__equatorial_coordinate_geodesic()
        
        substitute(self._proper_geodesic, self._k_to_a)
        substitute(self._equatorial_proper_geodesic, self._k_to_a)
                                        
                                        
    def __proper_geodesic(self):
        self._proper_geodesic := K^{\mu} = \Gamma^{\mu}_{\alpha \beta} u^{\alpha} u^{\beta}.
        substitute(self._proper_geodesic, self._chris)
        evaluate(self._proper_geodesic, self._u, rhsonly=True)
        
    def __coordinate_geodesic(self):
        self._coordinate_geodesic := a^{\mu} = -K^{\mu}+ K^{t} u^{\mu}.
        substitute(self._coordinate_geodesic, self._proper_geodesic)
        evaluate(self._coordinate_geodesic,self._u,rhsonly=True)
        substitute(self._coordinate_geodesic, self._ut1)
        
    def __equatorial_proper_geodesic(self):
        self._equatorial_proper_geodesic := K^{\mu} = \Gamma^{\mu}_{\alpha \beta} u^{\alpha} u^{\beta}.
        substitute(self._equatorial_proper_geodesic, self._chris)
        evaluate(self._equatorial_proper_geodesic, self._u, rhsonly=True)
        substitute(self._equatorial_proper_geodesic, self._ut1)
        substitute(self._equatorial_proper_geodesic,$\theta -> \pi /2$)
        map_sympy(self._equatorial_proper_geodesic, "simplify")
        substitute(self._equatorial_proper_geodesic, $u^{\theta} -> 0$)
        substitute(self._equatorial_proper_geodesic,$\pi-> 2 \theta$)
    
    def __equatorial_coordinate_geodesic(self):
        self._equatorial_coordinate_geodesic := a^{\mu} = -K^{\mu}+ K^{t} u^{\mu}.
        substitute(self._equatorial_coordinate_geodesic, self._proper_geodesic)
        evaluate(self._equatorial_coordinate_geodesic,self._u,rhsonly=True)
        substitute(self._equatorial_coordinate_geodesic, self._ut1)
        substitute(self._equatorial_coordinate_geodesic,$\theta -> \pi /2$)
        map_sympy(self._equatorial_coordinate_geodesic, "simplify")
        substitute(self._equatorial_coordinate_geodesic, $u^{\theta} -> 0$)
        substitute(self._equatorial_coordinate_geodesic,$\pi-> 2 \theta$)
    
    
    
    def geodesic_component(self, geodesic, component):     
        return get_component(geodesic, component)


    def save_geodesic(self, geodesic, filename):
        dv = [0,0,0,0]
        jj = 0
        for i in [Ex("t"),Ex("r"), Ex("\\theta"), Ex("\\phi")]:
            a = get_rhs(get_component(geodesic, i))
            b = substitute(a, self._indice_to_number)._sympy_();
            dv[jj] = b
            jj += 1

        with open(filename+".pkl", "wb") as file:
           for i in range(4):
               pickle.dump(dv[i], file)
           


In [5]:
schw = Met(metric)

${}\left[g_{t t} = -1+2M {r}^{-1},~ g_{r r} = {\left(1-2M {r}^{-1}\right)}^{-1},~ g_{\theta \theta} = {r}^{2},~ g_{\phi \phi} = {r}^{2} {\left(\sin{\theta}\right)}^{2},~ g^{t t} = {\left(2M {r}^{-1}-1\right)}^{-1},~ g^{r r} = -2M {r}^{-1}+1,~ g^{\phi \phi} = {\left({r}^{2} {\left(\sin{\theta}\right)}^{2}\right)}^{-1},~ g^{\theta \theta} = {r}^{-2}\right]$